<a href="https://colab.research.google.com/github/crew-guy/ml-with-tensorflow/blob/main/introduction-to-tensors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In this notebook, we are going to cover some of the most fundamental concepts of tensors using TensorFlow

* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & NumPy
* Using @tf.function (a way to speed up Python functions)
* Using GPUs with tensorflow (or TPUs)

## Introduction to tensors

In [ ]:
# Import tensorflow

import tensorflow as tf
print(tf.__version__)


2.9.2


In [ ]:
 # Create a tensor from a tensor like object
 scalar = tf.constant(7)
 print(scalar)

tf.Tensor(7, shape=(), dtype=int32)


In [ ]:
# Check the number of dimensions of a tensor
print(scalar.ndim)  

0


In [ ]:
# Create a vector
vector = tf.constant([10,10])
print(vector)

tf.Tensor([10 10], shape=(2,), dtype=int32)


In [ ]:
# Create a matrix
matrix = tf.constant([[1,2],[3,4]])
    print(matrix, matrix.ndim)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32) 2


In [ ]:
# Specify the data type with the dtype parameter
matrix2 = tf.constant([
    [1.,2.],
    [3.,4.],
    [5.,6.]
    ], dtype = tf.float16)
print(matrix2, matrix2.ndim)

tf.Tensor(
[[1. 2.]
 [3. 4.]
 [5. 6.]], shape=(3, 2), dtype=float16) 2


 ## What we've created so far
Scalar: A constant number
<br/>
Vector: A array of numbers representing a magnitude with a direction (eg: wind speed)
<br/>
Matrix: A 2-D array of numbers
<br/>
Tensor: An n-dimensional array of numbers


## Creating tensors withh `tf.Variable`

In [ ]:
changeable_tensor = tf.Variable([10,7])
unchangeable_tensor = tf.constant([10,7])

changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [ ]:
# Values inside a changeable_tensor (created with tf.Variable) can be reassigned whereas the same cannot be done for unchangeable_tensor

changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

🔑 Note : Rarely in practice, we decide, whether to use tf.constant() or tf.Variable, as Tensorflow does this for use. However, if in doubt, use tf.constant and change it later

## Creating random tensors
Random tensors are tensors of some arbitrary size which contain random numbers



In [ ]:
# Create 2 random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42) # set seed for reproducibility
random_1 = random_1.normal(shape=(3,2))
random_1

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ]], dtype=float32)>

In [ ]:
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3,2))
random_1 == random_2

<tf.Tensor: shape=(3, 2), dtype=bool, numpy=
array([[ True,  True],
       [ True,  True],
       [ True,  True]])>

## Shuffle the order of elements in a tensor

Valuable when you want to shuffle your data so that inherent order does not affect learning

> Rule : "If want to make our tensor shuffling reproducible, then we have to specify global & local tensors"

In [ ]:
# Shuffling a tensor
not_shuffled = tf.Variable([[10,7],[6,5],[3,2]])

# Using random shuffle - shuffling happens along the outermost dimension, i.e. dimension-0
shuffled = tf.random.shuffle(not_shuffled)

shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 3,  2],
       [10,  7],
       [ 6,  5]], dtype=int32)>

In [ ]:
# Setting a seed for shuffling will allow reproducibity - https://www.tensorflow.org/api_docs/python/tf/random/set_seed
# Notion notes for understanding seed - https://www.notion.so/techsoc/Understanding-Seeds-29579a6f8b6c4f4a828cfef8629f09d6

# Setting "global seed"
tf.random.set_seed(22)


# Setting operation-level aka "local seed"
# Local seeds override global seeds
tf.random.shuffle(not_shuffled, seed= 21)


<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 6,  5],
       [10,  7],
       [ 3,  2]], dtype=int32)>

## Other ways of creating a tensor

In [ ]:
# Using tf.ones & tf.zeros

# We can pass the shape of the tensor as an array or a tuple

ones_t = tf.ones([3,4], dtype=tf.float16)
zeros_t = tf.zeros(shape=(3,3,2))

ones_t, zeros_t

(<tf.Tensor: shape=(3, 4), dtype=float16, numpy=
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float16)>,
 <tf.Tensor: shape=(3, 3, 2), dtype=float32, numpy=
 array([[[0., 0.],
         [0., 0.],
         [0., 0.]],
 
        [[0., 0.],
         [0., 0.],
         [0., 0.]],
 
        [[0., 0.],
         [0., 0.],
         [0., 0.]]], dtype=float32)>)

In [ ]:
# Creating tensor from NumPy array
# Only difference between numpy array & tensors is that tensor computations can be run on a GPU (much faster)

import numpy as np
numpy_A = np.random.randint(-4, 8, size=(3,4))

print(numpy_A)

tensor_A = tf.constant(numpy_A, shape=(2,3,2))


print(tensor_A)

[[ 3 -2 -3  5]
 [-3  6  5  7]
 [ 2 -2  3 -4]]
tf.Tensor(
[[[ 3 -2]
  [-3  5]
  [-3  6]]

 [[ 5  7]
  [ 2 -2]
  [ 3 -4]]], shape=(2, 3, 2), dtype=int64)


# Getting information from tensors

* Shape - The length (number of elements) of each of the dimensions of the tensor - `tensor1.shape`
* Rank - The number of tensor dimensions - `tensor1.ndim`
* Size - Total number of elements in the tensor - `tf.size(tensor1)`
* Axis/Dimension - A particular dimension of the tensor - `tensor1[AXIS_NUMBER]`

In [32]:
tensor_1 = tf.ones([2,3,4,5], dtype = tf.float16)

tensor_1, tensor_1.shape, tf.size(tensor_1), tensor_1.ndim

(<tf.Tensor: shape=(2, 3, 4, 5), dtype=float16, numpy=
 array([[[[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],
 
         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],
 
         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]]],
 
 
        [[[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],
 
         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],
 
         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]]]], dtype=float16)>,
 TensorShape([2, 3, 4, 5]),
 <tf.Tensor: shape=(), dtype=int32, numpy=120>,
 4)

In [36]:
# Get various attributes of our tensor - "pretty print method" - really cool snippet to actually get imp metadata of a tensor
print("Datatype of every element:", tensor_1.dtype)
print("Shape of the tensor:", tensor_1.shape)
print("Number of dimensions of the tensor:", tensor_1.ndim)
print("Elements along the 0 axis:", tensor_1.shape[0])
print("Elements along the last axis:", tensor_1.shape[-1])
print("Total number of elements in the tensor", tf.size(tensor_1))
print("Total number of elements in the tensor", tf.size(tensor_1).numpy())

Datatype of every element: <dtype: 'float16'>
Shape of the tensor: (2, 3, 4, 5)
Number of dimensions of the tensor: 4
Elements along the 0 axis: 2
Elements along the last axis: 5
Total number of elements in the tensor tf.Tensor(120, shape=(), dtype=int32)
Total number of elements in the tensor 120


## Indexing & expanding tensors

Tensors can be indexed just like Python lists

In [38]:
# Get the 1st 2 elements of each dimension

first_2_elems = tensor_1[:2,:2,:2,:2]

# Get all elements, except the final one, from each dimension 

except_last_dim = tensor_1[:-1,:-1,:-1,:-1]

# Get the 1st element from each dimension from each each index, except for the final one

a = tensor_1[:1,:1,:1,:]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float16, numpy=array([[[[1., 1., 1., 1., 1.]]]], dtype=float16)>

In [43]:
# For rank 2 tensors


rank_2_tensor = tf.constant([[10,7], [32,3]])
# Get the last item of each row of a rank 2 tensor

k = rank_2_tensor[:,-1]

k

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 3], dtype=int32)>

In [45]:
# Add in extra dimension to our rank-2 tensor
# The "..." notation helps specify all axes of the tensor we are adding an extra dimension to
rank_3_tensor = rank_2_tensor[..., tf.newaxis]

# Alternative method - manually specify all axes for the tensor we are mutating
# rank_3_tensor = rank_2_tensor[:,:, tf.newaxis]

rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[32],
        [ 3]]], dtype=int32)>

In [46]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis=-1) # "-1" means expand on the final axis

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[32],
        [ 3]]], dtype=int32)>

In [47]:
# Expanding along 0-th axis
tf.expand_dims(rank_2_tensor, axis=0)

<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[10,  7],
        [32,  3]]], dtype=int32)>

## Manipulating tensors (tensor operations)